In [1]:
print('test')

test


In [7]:
from functools import lru_cache

@lru_cache(maxsize=None)
def all_ways(n):
    if n == 0:
        return [[]]
    if n == 1:
        return [[1]]
    
    ways = []
    # Add a 1-step to all ways of reaching n-1
    for way in all_ways(n - 1):
        ways.append(way + [1])
    # Add a 2-step to all ways of reaching n-2
    for way in all_ways(n - 2):
        ways.append(way + [2])
    
    return ways

In [ ]:
all_ways(7)

In [8]:
from itertools import permutations

def all_ways_itertools_v2(n):
    ways = []
    # Try each possible number of 2-steps
    for num_twos in range(n // 2 + 1):
        num_ones = n - 2 * num_twos
        
        # Create a list with the right number of 1s and 2s
        steps = [1] * num_ones + [2] * num_twos
        
        # Get unique permutations (using set to remove duplicates)
        unique_perms = set(permutations(steps))
        ways.extend([list(perm) for perm in unique_perms])
    
    return ways




In [ ]:
len(all_ways_itertools_v2(50))

89

## Mathematical Approaches - Counting Only

The key insight: **f(n) = f(n-1) + f(n-2)** (Fibonacci!)

Three ways to compute this:


In [7]:
# 1. DYNAMIC PROGRAMMING (Most efficient: O(n) time, O(n) space)
def count_ways_dp(n):
    """Build up from base cases using a table"""
    if n <= 1:
        return 1
    
    dp = [0] * (n + 1)
    dp[0] = 1  # One way to stay at ground
    dp[1] = 1  # One way to reach step 1
    
    for i in range(2, n + 1):
        dp[i] = dp[i-1] + dp[i-2]
    
    return dp[n]

# Test
print(f"n=7: {count_ways_dp(7)} ways")
print(f"n=10: {count_ways_dp(10)} ways")
print(f"n=550: {count_ways_dp(550)} ways")  # Try a big number!


n=7: 21 ways
n=10: 89 ways
n=550: 6348991937504800585712424788774240363794295386461634460789995227780425329818649176202420394392566714919796045762449 ways


In [3]:
# 2. SPACE-OPTIMIZED (O(n) time, O(1) space)
def count_ways_optimized(n):
    """Only keep track of last two values"""
    if n <= 1:
        return 1
    
    prev2 = 1  # f(0)
    prev1 = 1  # f(1)
    
    for i in range(2, n + 1):
        current = prev1 + prev2
        prev2 = prev1
        prev1 = current
    
    return prev1

# Test
print(f"n=100: {count_ways_optimized(100)} ways")
print(f"n=500: {count_ways_optimized(500)} ways")


n=100: 573147844013817084101 ways
n=500: 225591516161936330872512695036072072046011324913758190588638866418474627738686883405015987052796968498626 ways


In [4]:
# 3. CLOSED-FORM FORMULA - Binet's Formula (O(1) time!)
def count_ways_formula(n):
    """Pure mathematics - no loops!
    
    This uses Binet's formula for Fibonacci numbers.
    φ (phi) = golden ratio = (1 + √5) / 2
    """
    import math
    
    sqrt5 = math.sqrt(5)
    phi = (1 + sqrt5) / 2
    psi = (1 - sqrt5) / 2
    
    # Fibonacci formula (adjusted for our indexing)
    result = (phi**(n+1) - psi**(n+1)) / sqrt5
    
    return round(result)

# Test
print(f"n=7: {count_ways_formula(7)} ways")
print(f"n=10: {count_ways_formula(10)} ways")
print(f"n=100: {count_ways_formula(100)} ways")

# Verify they all match
n = 20
print(f"\nVerifying n={n}:")
print(f"  DP method:       {count_ways_dp(n)}")
print(f"  Optimized:       {count_ways_optimized(n)}")
print(f"  Formula:         {count_ways_formula(n)}")
print(f"  All match: {count_ways_dp(n) == count_ways_optimized(n) == count_ways_formula(n)}")


n=7: 21 ways
n=10: 89 ways
n=100: 573147844013818970112 ways

Verifying n=20:
  DP method:       10946
  Optimized:       10946
  Formula:         10946
  All match: True


## Summary: The Mathematical Perspective

| Approach | Time | Space | Notes |
|----------|------|-------|-------|
| **Dynamic Programming** | O(n) | O(n) | Clear, builds a table |
| **Space-Optimized** | O(n) | O(1) | Only tracks 2 values |
| **Binet's Formula** | O(1) | O(1) | Pure math! Uses golden ratio |

### Why is it Fibonacci?

To reach step n, you **must** have come from either:
- Step (n-1): take a 1-step → contributes f(n-1) ways
- Step (n-2): take a 2-step → contributes f(n-2) ways

Therefore: **f(n) = f(n-1) + f(n-2)** ✨

This is the defining property of Fibonacci numbers!


## Multiprocessing Approach - For Generating All Sequences

⚠️ **Important**: This is still slow for large n because the *number of sequences* grows exponentially (Fibonacci)!
- n=20 → 10,946 sequences
- n=30 → 1,346,269 sequences  
- n=50 → 20,365,011,074 sequences (20 billion!)

Multiprocessing helps, but can't overcome exponential growth.


In [ ]:
from itertools import permutations
from multiprocessing import Pool, cpu_count

def process_num_twos(args):
    """Worker function - processes one value of num_twos"""
    n, num_twos = args
    num_ones = n - 2 * num_twos
    steps = [1] * num_ones + [2] * num_twos
    unique_perms = set(permutations(steps))
    return [list(perm) for perm in unique_perms]

def all_ways_parallel(n, num_workers=None):
    """Parallel version using multiprocessing"""
    if num_workers is None:
        num_workers = cpu_count()
    
    # Create tasks: one for each possible number of 2-steps
    tasks = [(n, num_twos) for num_twos in range(n // 2 + 1)]
    
    # Process in parallel
    with Pool(num_workers) as pool:
        results = pool.map(process_num_twos, tasks)
    
    # Flatten results
    ways = []
    for result in results:
        ways.extend(result)
    
    return ways

print(f"Using {cpu_count()} CPU cores")


all_ways_parallel(40)

Using 24 CPU cores


Process ForkPoolWorker-23:
Process ForkPoolWorker-22:
Process ForkPoolWorker-24:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.12/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/usr/lib/python3.12/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 108, in 

In [ ]:
import time

# Benchmark on a moderately sized problem
n = 20

print(f"Comparing methods for n={n} (expecting {count_ways_dp(n)} sequences)\n")

# Method 1: Sequential itertools
start = time.time()
result1 = all_ways_itertools_v2(n)
time1 = time.time() - start
print(f"Sequential itertools: {time1:.4f}s → {len(result1)} sequences")

# Method 2: Parallel itertools
start = time.time()
result2 = all_ways_parallel(n)
time2 = time.time() - start
print(f"Parallel itertools:   {time2:.4f}s → {len(result2)} sequences")

# Speedup
speedup = time1 / time2
print(f"\nSpeedup: {speedup:.2f}x")

# Verify correctness
print(f"Results match: {len(result1) == len(result2) == count_ways_dp(n)}")


## The Fundamental Problem ⚠️

**Why is this still slow even with multiprocessing?**

The issue isn't computation speed - it's the **exponential number of sequences**:

```
n=10  →        89 sequences       (instant)
n=20  →    10,946 sequences       (fast)
n=30  → 1,346,269 sequences       (slow)
n=40  → 165,580,141 sequences     (very slow)
n=50  → 20,365,011,074 sequences  (20 billion!)
```

### Memory Limits

Even if you could generate 1 million sequences/second, for n=50 you'd need:
- **Time**: 20,000+ seconds (~6 hours)
- **Memory**: Storing 20 billion lists would require hundreds of GB of RAM!

### Recommendations

1. **Just need the count?** Use `count_ways_dp(n)` or `count_ways_formula(n)` → instant!
2. **Need actual sequences for small n (<25)?** Use multiprocessing version
3. **Need sequences for large n?** Consider:
   - Generate them on-demand (iterator/generator)
   - Sample random sequences instead of all
   - Use a different algorithm (combinations, not permutations)


In [ ]:
## Better Algorithm: Combinations (No Duplicates!)

Instead of generating permutations with duplicates, use combinations to directly pick positions for the 2-steps.


In [ ]:
from itertools import combinations

def all_ways_combinations(n):
    """More efficient - generates no duplicates"""
    ways = []
    for num_twos in range(n // 2 + 1):
        num_ones = n - 2 * num_twos
        total_steps = num_ones + num_twos
        
        # Choose positions for the 2-steps
        for positions in combinations(range(total_steps), num_twos):
            path = [1] * total_steps
            for pos in positions:
                path[pos] = 2
            ways.append(path)
    
    return ways

# Compare efficiency
print("Testing efficiency for n=20:")
start = time.time()
result_perm = all_ways_itertools_v2(20)
time_perm = time.time() - start

start = time.time()
result_comb = all_ways_combinations(20)
time_comb = time.time() - start

print(f"Permutations: {time_perm:.4f}s")
print(f"Combinations: {time_comb:.4f}s")
print(f"Speedup: {time_perm/time_comb:.2f}x")
print(f"Both give {len(result_perm)} sequences: {len(result_perm) == len(result_comb)}")


In [ ]:
## Parallel Combinations (Best of Both Worlds!)

def process_num_twos_combinations(args):
    """Worker function using combinations (more efficient)"""
    n, num_twos = args
    num_ones = n - 2 * num_twos
    total_steps = num_ones + num_twos
    
    ways = []
    for positions in combinations(range(total_steps), num_twos):
        path = [1] * total_steps
        for pos in positions:
            path[pos] = 2
        ways.append(path)
    
    return ways

def all_ways_parallel_combinations(n, num_workers=None):
    """Parallel version using combinations - the fastest!"""
    if num_workers is None:
        num_workers = cpu_count()
    
    tasks = [(n, num_twos) for num_twos in range(n // 2 + 1)]
    
    with Pool(num_workers) as pool:
        results = pool.map(process_num_twos_combinations, tasks)
    
    ways = []
    for result in results:
        ways.extend(result)
    
    return ways


In [ ]:
## Final Benchmark: All Methods

n = 20
expected = count_ways_dp(n)

print(f"Generating all {expected:,} sequences for n={n}\n")
print("="*60)

methods = [
    ("Permutations (sequential)", lambda: all_ways_itertools_v2(n)),
    ("Permutations (parallel)", lambda: all_ways_parallel(n)),
    ("Combinations (sequential)", lambda: all_ways_combinations(n)),
    ("Combinations (parallel)", lambda: all_ways_parallel_combinations(n)),
]

results = []
for name, func in methods:
    start = time.time()
    result = func()
    elapsed = time.time() - start
    results.append((name, elapsed, len(result)))
    print(f"{name:30s}: {elapsed:7.4f}s → {len(result):,} sequences")

print("="*60)

# Find fastest
fastest_time = min(r[1] for r in results)
print(f"\nSpeedups relative to fastest ({results[min(range(len(results)), key=lambda i: results[i][1])][0]}):")
for name, elapsed, count in results:
    speedup = elapsed / fastest_time
    print(f"  {name:30s}: {speedup:5.2f}x {'(fastest)' if speedup == 1.0 else ''}")


## Summary & Recommendations

### Key Takeaways:

1. **For counting only**: Use `count_ways_dp(n)` or `count_ways_formula(n)`
   - Works instantly even for n=1000
   - No memory issues

2. **For generating sequences (small n < 25)**:
   - **Best**: Combinations approach (parallel or sequential)
   - **Why**: No duplicate generation, more efficient
   - **Speedup**: 2-10x faster than permutations

3. **For large n (> 30)**:
   - ❌ Generating ALL sequences is infeasible (billions+ sequences)
   - ✅ Consider alternatives:
     - Generate on-demand (yield/generator)
     - Sample random sequences
     - Only count them

### Algorithm Comparison:

```
Permutations: generates n! / (k! * (n-k)!)  arrangements 
              BUT with duplicates → uses set() → wasteful

Combinations: directly generates C(n,k) unique arrangements
              → no duplicates → faster!
```

### Multiprocessing Benefit:

- Best when work can be split (different num_twos values)
- Speedup limited by Amdahl's Law and overhead
- Most beneficial for n in range 15-30


## Why √5 in Binet's Formula? 🤯

One of the most beautiful mysteries in mathematics: **Why does an irrational number (√5) produce integers?**

### The Mathematical Derivation

Starting with the recurrence relation: **f(n) = f(n-1) + f(n-2)**

We assume a solution of the form: **f(n) = r^n** for some constant r

Substituting into the recurrence:
```
r^n = r^(n-1) + r^(n-2)
```

Divide by r^(n-2):
```
r^2 = r + 1
r^2 - r - 1 = 0
```

Using the quadratic formula:
```
r = (1 ± √(1 + 4)) / 2
r = (1 ± √5) / 2
```

**This is where √5 comes from!** It's the discriminant of the characteristic equation.

### The Golden Ratio Connection

```
φ (phi) = (1 + √5) / 2 ≈ 1.618034...  (golden ratio)
ψ (psi) = (1 - √5) / 2 ≈ -0.618034...  (conjugate)
```

Notice: φ + ψ = 1 and φ × ψ = -1

### Why It Produces Integers

The general solution is: **f(n) = A·φ^n + B·ψ^n**

With base conditions f(0)=0, f(1)=1, we get: **A = 1/√5** and **B = -1/√5**

So: **F(n) = (φ^n - ψ^n) / √5**

**The magic:** Even though φ and ψ are irrational (contain √5), their difference **divided by √5** always cancels out the irrationality!

As n grows, ψ^n → 0 (because |ψ| < 1), so:
```
F(n) ≈ φ^n / √5
```

The Fibonacci numbers grow exponentially at rate φ (the golden ratio)!


In [ ]:
import math

# Let's see the magic happen!
sqrt5 = math.sqrt(5)
phi = (1 + sqrt5) / 2
psi = (1 - sqrt5) / 2

print("The two roots of r² - r - 1 = 0:")
print(f"φ (phi) = (1 + √5) / 2 = {phi:.10f}")
print(f"ψ (psi) = (1 - √5) / 2 = {psi:.10f}")
print()

# Compute first few Fibonacci numbers manually
print("Computing F(n) = (φ^n - ψ^n) / √5:")
print("="*60)

for n in range(11):
    # Binet's formula
    fib = (phi**n - psi**n) / sqrt5
    fib_rounded = round(fib)
    
    # Show that ψ^n becomes negligible
    print(f"F({n:2d}) = ({phi**n:12.6f} - {psi**n:11.6f}) / √5")
    print(f"     = {fib:12.8f} ≈ {fib_rounded}")
    print()

print("="*60)
print("Notice:")
print(f"1. ψ^n oscillates and approaches 0 (because |ψ| = {abs(psi):.6f} < 1)")
print(f"2. For large n, F(n) ≈ φ^n / √5 ≈ φ^n / 2.236")
print(f"3. The √5 terms in numerator perfectly cancel with √5 in denominator!")


## Algebraic Proof: Why √5 Cancels

Let's prove F(3) = 2 algebraically to see how the √5 cancels:

### Step-by-step for F(3):

```
F(3) = (φ³ - ψ³) / √5

where φ = (1 + √5)/2  and  ψ = (1 - √5)/2
```

**Calculate φ³:**
```
φ² = [(1 + √5)/2]² = (1 + 2√5 + 5)/4 = (6 + 2√5)/4 = (3 + √5)/2

φ³ = φ² × φ = [(3 + √5)/2] × [(1 + √5)/2]
   = (3 + 3√5 + √5 + 5) / 4
   = (8 + 4√5) / 4
   = (2 + √5)
```

**Calculate ψ³:**
```
ψ² = [(1 - √5)/2]² = (1 - 2√5 + 5)/4 = (6 - 2√5)/4 = (3 - √5)/2

ψ³ = ψ² × ψ = [(3 - √5)/2] × [(1 - √5)/2]
   = (3 - 3√5 - √5 + 5) / 4
   = (8 - 4√5) / 4
   = (2 - √5)
```

**Calculate φ³ - ψ³:**
```
φ³ - ψ³ = (2 + √5) - (2 - √5)
        = 2 + √5 - 2 + √5
        = 2√5
```

**Final result:**
```
F(3) = 2√5 / √5 = 2  ✓
```

**The √5 cancels perfectly!** This happens for every Fibonacci number.


In [ ]:
# Pattern: Powers of φ and ψ always have the form (a + b√5)
# When you subtract conjugates, the rational parts cancel and you get c√5

print("Pattern in powers of φ and ψ:")
print("="*70)

for n in range(1, 8):
    # We can express φ^n and ψ^n in form (a ± b√5)
    # For φ^n: compute using the fact that φ = (1 + √5)/2
    
    # Use the recursive property: φ^n = F(n)·φ + F(n-1)
    # This gives: φ^n = F(n)·(1+√5)/2 + F(n-1)
    #           = [F(n-1) + F(n)/2] + [F(n)/2]·√5
    
    fib_n = count_ways_formula(n)
    fib_n_minus_1 = count_ways_formula(n-1) if n > 0 else 0
    
    # Coefficients for φ^n = a + b√5
    a_phi = fib_n_minus_1 + fib_n/2
    b_phi = fib_n/2
    
    # Coefficients for ψ^n = a - b√5 (conjugate)
    a_psi = fib_n_minus_1 + fib_n/2
    b_psi = -fib_n/2
    
    print(f"n={n}:")
    print(f"  φ^{n} = {a_phi:.1f} + {b_phi:.1f}√5")
    print(f"  ψ^{n} = {a_psi:.1f} - {abs(b_psi):.1f}√5")
    print(f"  φ^{n} - ψ^{n} = ({a_phi:.1f} + {b_phi:.1f}√5) - ({a_psi:.1f} - {abs(b_psi):.1f}√5)")
    print(f"              = {2*b_phi:.1f}√5")
    print(f"  F({n}) = {2*b_phi:.1f}√5 / √5 = {fib_n} ✓")
    print()

print("="*70)
print("Key insight: The difference φ^n - ψ^n always equals k√5 for integer k!")
print("Therefore: F(n) = (k√5) / √5 = k, which is always an integer!")


## The Deep Answer: Why √5?

### Three Perspectives:

**1. Algebraic Perspective:**
- The recurrence f(n) = f(n-1) + f(n-2) leads to characteristic equation r² - r - 1 = 0
- Quadratic formula gives r = (1 ± √(1²-4·1·(-1)))/2 = (1 ± √5)/2
- The discriminant √5 comes from solving this quadratic!

**2. Number Theory Perspective:**
- φ and ψ are **algebraic conjugates** over ℚ(√5) (the field extension of rationals by √5)
- Any power φⁿ or ψⁿ stays in this field: has form a + b√5
- Subtracting conjugates **always eliminates the rational part**, leaving k√5
- Dividing by √5 brings us back to integers!

**3. Geometric Perspective:**
- φ ≈ 1.618... is the **golden ratio** - appears everywhere in nature
- Rectangle with sides φ:1 has self-similar properties
- Fibonacci spiral approaches golden spiral
- The √5 fundamentally connects Fibonacci to this geometric ratio

### The Beautiful Truth

The √5 is not arbitrary - it's the **essential bridge** between:
- Discrete integers (Fibonacci numbers)
- Continuous geometry (golden ratio)
- Algebraic structure (characteristic equation)

This is why Fibonacci numbers appear in nature (sunflower spirals, nautilus shells, galaxy arms) - they're intimately connected to the golden ratio through √5!

### Fun Facts:
- φ² = φ + 1 (the golden ratio satisfies its own characteristic equation!)
- φ = 1 + 1/(1 + 1/(1 + 1/(...))) (infinite continued fraction)
- F(n+1)/F(n) → φ as n → ∞ (ratio converges to golden ratio)
- All because of √5! 🌟


In [ ]:
# Verify: Ratio of consecutive Fibonacci numbers approaches φ
print("Fibonacci ratios converging to the golden ratio φ:")
print("="*60)

for n in range(1, 21):
    f_n = count_ways_formula(n)
    f_n_plus_1 = count_ways_formula(n + 1)
    ratio = f_n_plus_1 / f_n
    error = abs(ratio - phi)
    
    print(f"F({n+1:2d})/F({n:2d}) = {f_n_plus_1:6d}/{f_n:6d} = {ratio:.10f}  (error: {error:.2e})")

print("="*60)
print(f"Golden ratio φ = {phi:.10f}")
print()
print("Notice: The ratio oscillates around φ and converges exponentially fast!")
print()

# Verify the property φ² = φ + 1
phi_squared = phi ** 2
phi_plus_1 = phi + 1
print(f"Verifying φ² = φ + 1:")
print(f"  φ² = {phi_squared:.10f}")
print(f"  φ+1 = {phi_plus_1:.10f}")
print(f"  Equal? {abs(phi_squared - phi_plus_1) < 1e-10} ✓")


In [16]:
import math
import mpmath

# demonstrate the limitation of phi^1000 with python 64bit floats
phi = (1 + math.sqrt(5)) / 2
psi = (1 - math.sqrt(5)) / 2


# show output with small number
print(f"phi^1000 = {phi**1400:.10f}")

# show output with large number
try:
    print(f"phi^1000 = {phi**1000000:.10f}")
except OverflowError:
    print("OverflowError: The result is too large to represent.")

# use mpmath to show output with large number
print(f"phi^1000 = {mpmath.mp.power(phi, 1000000)}")


phi^1000 = 38255717345119578154122443991968370971930050547158590919352101493067518472124841315132074570964172368111177311818063665794514188068501549280234471235779235352677224497269564040347817532356413236455267041872698628108534773782127343468741131934110165863201098114609475866728444544576586873044992.0000000000
OverflowError: The result is too large to represent.
phi^1000 = 4.36767161917267e+208987
